## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


### BUSINESS UNDERSTANDING

##### KEY QUESTIONS FOR MOVIE STUDIO STRATEGY
1. Which movie genres generate the highest revenue? By analyzing the total earnings from the different genres, we can determine which types of films are more profitable and likely to attract large audiences.
2. Does movie rating or critic score correlate with higher box office success? using ratings from Rotten Tomatoes and movie database, we can measure whether higher-rated movies earn more revenue 
3. Does a longer runtime affect rating or revenue? By analyzing the runtimes in the dataset, we can identify which runtimes generate higher earnings for different types of movies.
4. Does the release timing affect the earnings? By examining which months and their associated revenue, we can identify the optimal times to launch films for maximum success.

#### Importing python libraries

In [55]:
import pandas as pd
import sqlite3


#### Checking tables in the imdb database

In [ ]:
# Connect to your IMDB database
conn = sqlite3.connect('im.db')
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

Tables in the database: []


#### Checking the imdb database file path and size

In [86]:
import os
print(os.path.abspath('im.db'))
print(os.path.getsize('im.db'))

c:\Users\Dell\Documents\flatiron\DataScience\Phase2\project2\Movie-Studio-EDA-Project\im.db
0
